# 1. Libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import json

import torch
from torch.distributions import Categorical
from transformers import AutoTokenizer, AutoModelForSequenceClassification

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


Mounted at /content/gdrive


# 2. Set up

In [2]:
train_data_path = "/content/gdrive/MyDrive/Colab Notebooks/MNLP/Homework I - task 4/binary/train.jsonl"
dev_data_path = "/content/gdrive/MyDrive/Colab Notebooks/MNLP/Homework I - task 4/binary/dev.jsonl"
test_data_path = "/content/gdrive/MyDrive/Colab Notebooks/MNLP/Homework I - task 4/gold_truth/binary/test.jsonl"

In [3]:
def reformat_jsonl(input_path, output_path):
  id_seq = 0
  with open(input_path, "r") as input_file, open(output_path, "w") as output_file:
    for line in input_file:
      json_data = json.loads(line)
      reformatted_json_data = {
          "id" : id_seq,
          "lemma" : json_data["lemma"],
          "sentence1" : json_data["sentence1"],
          "sentence2" : json_data["sentence2"],
          "start1" : json_data["start1"],
          "end1" : json_data["end1"],
          "start2" : json_data["start2"],
          "end2" : json_data["end2"],
          "choices" : ["Ambiguo", "Non ambiguo"],
          "label" : json_data["label"]
      }
      json.dump(reformatted_json_data, output_file)
      output_file.write('\n')
      id_seq += 1

def read_lines_jsonl(file_path, num_lines):
  with open(file_path, 'r') as f:
    json_list = list(f)
    for line in json_list[:num_lines]:
      data = json.loads(line)
      print(data)

In [4]:
# italian train data
reformat_jsonl(train_data_path, "WiC-ITA_train.jsonl")
read_lines_jsonl("WiC-ITA_train.jsonl", num_lines = 10) # preview of the first 10 lines

{'id': 0, 'lemma': 'affare', 'sentence1': 'E visto che mi lascia GPS , Ecoscandaglio , TUTTE le dotazioni obbligatorie in ottimo stato ( razzi nuovi , ecc ) un sacco di cime ( due ancora nuove imballate ) , gonfiatore , pezzi vari di ricambio , ecc ecc ecc con un supplemento è affare fatto .', 'sentence2': 'La rivista nordamericana segnala come presunti sospetti il milionario e potente della stampa venezuelana , Gustavo Cisnero , che avrebbe ricevuto istruzioni da Otto Reich , massimo responsabile della Casa Bianca per gli Affari Latinoamericani .', 'start1': 235, 'end1': 241, 'start2': 220, 'end2': 226, 'choices': ['Ambiguo', 'Non ambiguo'], 'label': 0}
{'id': 1, 'lemma': 'elemento', 'sentence1': "In particolare , l' elemento più scandaloso del provvedimento che ci accingiamo a votare è rappresentato dagli articoli 7 e 8 su cui , questa mattina , abbiamo condotto , come opposizione , una lunga battaglia di buon senso .", 'sentence2': 'Il Fondo mostra di apprezzare " certi elementi di 

In [5]:
# italian test data
reformat_jsonl(test_data_path, "WiC-ITA_test.jsonl")
read_lines_jsonl("WiC-ITA_test.jsonl", num_lines = 10) # preview of the first 10 lines

{'id': 0, 'lemma': 'minore', 'sentence1': "Dopo innumerevoli provini sono stati scelti 15 attori per i ruoli principali , 200 per le parti minori e quasi 4000 comparse che animeranno le 24 puntate da 50 minuti ognuna trasmesse ogni domenica con due episodi l' uno di seguito all' altro .", 'sentence2': 'Il direttore di Micromega ha detto che , dopo questi due giorni , ci troviamo di fronte alla scelta fra un bene e un male minore .', 'start1': 96, 'end1': 102, 'start2': 122, 'end2': 128, 'choices': ['Ambiguo', 'Non ambiguo'], 'label': 1}
{'id': 1, 'lemma': 'rigore', 'sentence1': "44 ' st Napoli vicinissimo al gol con Floro Flores lanciato in area di rigore da un colpo di testa di Stellone : sull' incursione dell' attaccante napoletano Casazza con un' uscita disperata riesce a respingere in angolo .", 'sentence2': "Nell' altra semifinale i simpaticissimi e tecnicamente validissimi ragazzi di Torino avevano la meglio ai calci di rigore della sezione di Roma 2 , dopo che i tempi regolament

In [6]:
# italian dev data
reformat_jsonl(dev_data_path, "WiC-ITA_dev.jsonl")
read_lines_jsonl("WiC-ITA_dev.jsonl", num_lines = 10) # preview of the first 10 lines

{'id': 0, 'lemma': 'delicatezza', 'sentence1': 'Il presidente AZZOLLINI , stante la delicatezza dei temi oggetto del provvedimento , ritiene opportuno richiedere la predisposizione di una relazione tecnica .', 'sentence2': 'Montare la panna senza zucchero in modo che diventi molto solida , quindi unirla al composto avendo cura di mescolarla con molta delicatezza .', 'start1': 36, 'end1': 47, 'start2': 129, 'end2': 140, 'choices': ['Ambiguo', 'Non ambiguo'], 'label': 0}
{'id': 1, 'lemma': 'fede', 'sentence1': "Per quanto riguarda gli adolescenti l' indagine rileva che la fede risulta più diffusa tra le ragazze : il 76,4 % delle adolescenti è credente , contro il 71,1 % dei coetanei di sesso maschile .", 'sentence2': 'Quando un datore di lavoro viene perseguito in ottemperanza alle disposizioni adottate in virtù del presente articolo , egli deve avere il diritto di produrre la prova della propria buona fede .', 'start1': 62, 'end1': 66, 'start2': 188, 'end2': 192, 'choices': ['Ambiguo', 

# 3. Prompt formulation

In [7]:
prompts = [
    "Il significato della parola '{lemma}' è ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'?",
    "Nella frase '{sentence1}' e nella frase '{sentence2}', il significato della parola '{lemma}' risulta ambiguo o non ambiguo?",
    "Data la parola '{lemma}', valuta se il suo significato risulta essere ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'."
]

In [8]:
def save_prompts_jsonl(prompts, file_path):
  json_prompts = []
  for prompt in prompts:
    json_prompts.append({"prompt": prompt})

  with open(file_path, "w") as output_file:
    for json_prompt in json_prompts:
      json.dump(json_prompt, output_file)
      output_file.write("\n")

In [9]:
save_prompts_jsonl(prompts, "prompts.jsonl")
read_lines_jsonl("prompts.jsonl", num_lines = 3)

{'prompt': "Il significato della parola '{lemma}' è ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'?"}
{'prompt': "Nella frase '{sentence1}' e nella frase '{sentence2}', il significato della parola '{lemma}' risulta ambiguo o non ambiguo?"}
{'prompt': "Data la parola '{lemma}', valuta se il suo significato risulta essere ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'."}


# 4. Performance Evaluation

## 4.1 Llama 2

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [11]:
model_id = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [75]:
def evaluate_prompt(prompt, data_file):
  formatted_prompts, y_true, y_pred = [], [], []

  with open(data_file, "r") as f:
    json_data = list(f)
    for line in json_data:
      pair = json.loads(line)
      id = pair['id']
      lemma = pair['lemma']
      sentence1 = pair['sentence1']
      sentence2 =  pair['sentence2']
      start1 = pair['start1']
      end1 = pair['end1']
      start2 = pair['start2']
      end2 = pair['end2']
      choices = pair['choices']
      label = pair['label']

      formatted_prompt = prompt.format(lemma = lemma, sentence1 = sentence1, sentence2 = sentence2)
      inputs = tokenizer(formatted_prompt, return_tensors = "pt").to(device)
      logits = model(**inputs).logits[0].detach().cpu().numpy()
      probabilities = torch.softmax(torch.tensor(logits), dim = -1).detach().cpu().numpy()
      predicted_label = 1 if probabilities[1] > probabilities[0] else 0

      formatted_prompts.append(formatted_prompt)
      y_true.append(label)
      y_pred.append(predicted_label)

    return formatted_prompts, y_true, y_pred


def visualize_results(results, num_results):
  choices = ["Ambiguo", "Non Ambiguo"]
  for prompt in range(len(results)):
    for n in range(num_results):
      print("Prompt: ", results[prompt][0][n])
      print("True label: ", choices[results[prompt][1][n]])
      print("Predicted label: ", choices[results[prompt][2][n]])
      print(" ")


def compute_metrics(y_true, y_pred):
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)
  cf_matrix = confusion_matrix(y_true, y_pred)

  return accuracy, precision, recall, f1, cf_matrix


def visualize_metrics(accuracy, precision, recall, f1, cf_matrix):
  print("Overall Accuracy: ", round(accuracy, 3))
  print("Overall Precision: ", round(precision, 3))
  print("Overall Recall: ", round(recall, 3))
  print("Overall F1-Score: ", round(f1, 3))

  # # Confusion Matrix Plot
  # fig, ax = plt.subplots(figsize=(10, 5))
  # sns.heatmap(cf_matrix, annot=True)
  # ax.set_title('Confusion Matrix')
  # ax.set_xlabel('Predicted Labels')
  # ax.set_ylabel('True Labels')
  # plt.show()

In [59]:
train_results = []
test_results = []
dev_results = []

for prompt in prompts:
  formatted_prompt, y_true, y_pred = evaluate_prompt(prompt, "WiC-ITA_train.jsonl")
  train_results.append([formatted_prompt, y_true, y_pred])

  formatted_prompt, y_true, y_pred = evaluate_prompt(prompt, "WiC-ITA_test.jsonl")
  test_results.append([formatted_prompt, y_true, y_pred])

  formatted_prompt, y_true, y_pred = evaluate_prompt(prompt, "WiC-ITA_dev.jsonl")
  dev_results.append([formatted_prompt, y_true, y_pred])

In [76]:
visualize_results(train_results, 2) # preview of the first 2 train results for each prompt

Prompt:  Il significato della parola 'affare' è ambiguo o non ambiguo nella frase 'E visto che mi lascia GPS , Ecoscandaglio , TUTTE le dotazioni obbligatorie in ottimo stato ( razzi nuovi , ecc ) un sacco di cime ( due ancora nuove imballate ) , gonfiatore , pezzi vari di ricambio , ecc ecc ecc con un supplemento è affare fatto .' e nella frase 'La rivista nordamericana segnala come presunti sospetti il milionario e potente della stampa venezuelana , Gustavo Cisnero , che avrebbe ricevuto istruzioni da Otto Reich , massimo responsabile della Casa Bianca per gli Affari Latinoamericani .'?
True label:  Ambiguo
Predicted label:  Non Ambiguo
 
Prompt:  Il significato della parola 'elemento' è ambiguo o non ambiguo nella frase 'In particolare , l' elemento più scandaloso del provvedimento che ci accingiamo a votare è rappresentato dagli articoli 7 e 8 su cui , questa mattina , abbiamo condotto , come opposizione , una lunga battaglia di buon senso .' e nella frase 'Il Fondo mostra di appre

In [63]:
train_metrics = []
test_metrics = []
dev_metrics = []

for prompt in range(len(prompts)):
  accuracy, precision, recall, f1, cf_matrix = compute_metrics(train_results[prompt][1], train_results[prompt][2])
  train_metrics.append([accuracy, precision, recall, f1, cf_matrix])

  accuracy, precision, recall, f1, cf_matrix = compute_metrics(test_results[prompt][1], test_results[prompt][2])
  test_metrics.append([accuracy, precision, recall, f1, cf_matrix])

  accuracy, precision, recall, f1, cf_matrix = compute_metrics(dev_results[prompt][1], dev_results[prompt][2])
  dev_metrics.append([accuracy, precision, recall, f1, cf_matrix])

## 4.2 Overall Statistics

In [64]:
# Train Statistics
for n in range(len(train_metrics)):
  print(f"Train Statistics for the Prompt {n}")
  print("Prompt: ", prompts[n])
  visualize_metrics(train_metrics[n][0], train_metrics[n][1], train_metrics[n][2], train_metrics[n][3], train_metrics[n][4])
  print(" ")

Train Statistics for the Prompt 0
Prompt:  Il significato della parola '{lemma}' è ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'?
Overall Accuracy:  0.687
Overall Precision:  0.708
Overall Recall:  0.954
Overall F1-Score:  0.813
 
Train Statistics for the Prompt 1
Prompt:  Nella frase '{sentence1}' e nella frase '{sentence2}', il significato della parola '{lemma}' risulta ambiguo o non ambiguo?
Overall Accuracy:  0.354
Overall Precision:  0.734
Overall Recall:  0.146
Overall F1-Score:  0.244
 
Train Statistics for the Prompt 2
Prompt:  Data la parola '{lemma}', valuta se il suo significato risulta essere ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'.
Overall Accuracy:  0.702
Overall Precision:  0.712
Overall Recall:  0.975
Overall F1-Score:  0.823
 


In [65]:
# Test Statistics
for n in range(len(train_metrics)):
  print(f"Test Statistics for the Prompt {n}")
  print("Prompt: ", prompts[n])
  visualize_metrics(test_metrics[n][0], test_metrics[n][1], test_metrics[n][2], test_metrics[n][3], test_metrics[n][4])
  print(" ")

Test Statistics for the Prompt 0
Prompt:  Il significato della parola '{lemma}' è ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'?
Overall Accuracy:  0.486
Overall Precision:  0.493
Overall Recall:  0.932
Overall F1-Score:  0.645
 
Test Statistics for the Prompt 1
Prompt:  Nella frase '{sentence1}' e nella frase '{sentence2}', il significato della parola '{lemma}' risulta ambiguo o non ambiguo?
Overall Accuracy:  0.49
Overall Precision:  0.464
Overall Recall:  0.128
Overall F1-Score:  0.201
 
Test Statistics for the Prompt 2
Prompt:  Data la parola '{lemma}', valuta se il suo significato risulta essere ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'.
Overall Accuracy:  0.492
Overall Precision:  0.496
Overall Recall:  0.96
Overall F1-Score:  0.654
 


In [66]:
# Dev Statistics
for n in range(len(dev_metrics)):
  print(f"Test Statistics for the Prompt {n}")
  print("Prompt: ", prompts[n])
  visualize_metrics(dev_metrics[n][0], dev_metrics[n][1], dev_metrics[n][2], dev_metrics[n][3], dev_metrics[n][4])
  print(" ")

Test Statistics for the Prompt 0
Prompt:  Il significato della parola '{lemma}' è ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'?
Overall Accuracy:  0.494
Overall Precision:  0.497
Overall Recall:  0.952
Overall F1-Score:  0.653
 
Test Statistics for the Prompt 1
Prompt:  Nella frase '{sentence1}' e nella frase '{sentence2}', il significato della parola '{lemma}' risulta ambiguo o non ambiguo?
Overall Accuracy:  0.512
Overall Precision:  0.539
Overall Recall:  0.164
Overall F1-Score:  0.252
 
Test Statistics for the Prompt 2
Prompt:  Data la parola '{lemma}', valuta se il suo significato risulta essere ambiguo o non ambiguo nella frase '{sentence1}' e nella frase '{sentence2}'.
Overall Accuracy:  0.488
Overall Precision:  0.494
Overall Recall:  0.964
Overall F1-Score:  0.653
 


# EXTRA ?


In [ ]:
# !pip install datasets
# !pip install accelerate -U
# !pip install transformers==4.30

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
train_dataset = load_dataset("json", data_files="WiC-ITA_train.jsonl")
dev_dataset = load_dataset("json", data_files="WiC-ITA_dev.jsonl")

def tokenize_dataset(dataset):
    tokenized_dataset = tokenizer(dataset["sentence1"], dataset["sentence2"], truncation=True, padding=True)
    tokenized_dataset["label"] = dataset["label"]
    return tokenized_dataset

train_dataset = train_dataset.map(tokenize_dataset)
dev_dataset = dev_dataset.map(tokenize_dataset)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset
)

trainer.train()